In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

In [3]:
import pandas as pd
import numpy as np
from collections import Counter
from copy import deepcopy
from sklearn import cross_validation

In [ ]:
import xgboost as xgb

In [4]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import roc_auc_score

In [3]:
data = pd.read_csv('./credit_train.csv', sep=';', parse_dates=True, dayfirst=True, encoding='cp1251', decimal = ',')

In [4]:
data2 = pd.read_csv('./credit_train.csv', sep=';', parse_dates=True, dayfirst=True, encoding='cp1251')

In [5]:
data.head()

,client_id,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,1,M,48,MAR,UMN,59998.00,10,1.6,0.770249,GRD,КРАСНОДАРСКИЙ КРАЙ,30000,1,1,0
1,2,F,28,MAR,UMN,10889.00,6,1.1,0.248514,GRD,МОСКВА,43000,2,0,0
2,3,M,32,MAR,SPC,10728.00,12,1.1,0.459589,SCH,ОБЛ САРАТОВСКАЯ,23000,5,0,0
3,4,F,27,DIV,SPC,12009.09,12,1.1,0.362536,GRD,ОБЛ ВОЛГОГРАДСКАЯ,17000,2,0,0
4,5,M,45,MAR,SPC,16908.89,10,1.1,0.421385,SCH,ЧЕЛЯБИНСКАЯ ОБЛАСТЬ,25000,1,0,0


In [113]:
data['living_region'][1100:1200]

1100                              ТАМБОВСКАЯ ОБЛ
1101                              ТАТАРСТАН РЕСП
1102                           РЕСП БАШКОРТОСТАН
1103                            ОРЕНБУРГСКАЯ ОБЛ
1104                        РЕСПУБЛИКА ТАТАРСТАН
1105                          КРАСНОДАРСКИЙ КРАЙ
1106                         ЧЕЛЯБИНСКАЯ ОБЛАСТЬ
1107                   РЕСП КАРАЧАЕВО-ЧЕРКЕССКАЯ
1108                           ОБЛ ЛЕНИНГРАДСКАЯ
1109                              РОСТОВСКАЯ ОБЛ
1110                             ОБЛ ВОРОНЕЖСКАЯ
1111                         ЯРОСЛАВСКАЯ ОБЛАСТЬ
1112                         РЕСП. САХА (ЯКУТИЯ)
1113                              ОБЛ МОСКОВСКАЯ
1114                           САНКТ-ПЕТЕРБУРГ Г
1115                           ИРКУТСКАЯ ОБЛАСТЬ
1116                        ОРЕНБУРГСКАЯ ОБЛАСТЬ
1117                               САМАРСКАЯ ОБЛ
1118                             САРАТОВСКАЯ ОБЛ
1119                              ОБЛ МОСКОВСКАЯ
1120                

In [6]:
Categorical_columns = ['gender', 'marital_status', 'job_position',  'education', 'living_region']
Numerical_columns = data.columns.drop(Categorical_columns)

In [7]:
cat_data = data.drop(Numerical_columns, axis = 1)
num_data = data.drop(Categorical_columns + ['client_id', 'tariff_id'], axis = 1)

In [8]:
num_data.columns

Index([u'age', u'credit_sum', u'credit_month', u'score_shk', u'monthly_income',
       u'credit_count', u'overdue_credit_count'],
      dtype='object')

In [9]:
# Количество NaN в числовых признаках
print('Количество NaN:')
nan_val = num_data.isnull().sum().T
nan_val

Количество NaN:


age                        0
credit_sum                 0
credit_month               0
score_shk                  0
monthly_income             1
credit_count            9230
overdue_credit_count    9230
dtype: int64

In [10]:
def fillNan(data, feature):
    data[[feature]] = data[[feature]].fillna(0)
    return data

In [11]:
num_data = fillNan(num_data, 'credit_count')
num_data = fillNan(num_data, 'overdue_credit_count')

In [12]:
income_null = num_data['monthly_income'].isnull()
for i in range(170000):
    if income_null[i]:
        print i
        break

19639


In [24]:
cat_data.values[19639]

array([19640, u'F', u'MAR', u'BIS', u'SCH',
       u'\u041f\u0420\u0418\u041c\u041e\u0420\u0421\u041a\u0418\u0419 \u041a\u0420\u0410\u0419'], dtype=object)

In [21]:
num_data['tariff_id'] = data2['tariff_id']

In [25]:
size = 170746

In [39]:
s = 0
count = 0
for i in range(size):
    if cat_data['job_position'][i] == 'BIS' and i != 19639 and cat_data['living_region'][i] == cat_data['living_region'][19639] :
        count +=1
        s += num_data['monthly_income'][i]

In [41]:
num_data['monthly_income'][19639] =  s / count

/root/miniconda/envs/rep_py2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [47]:
living_region = cat_data['living_region'].isnull()
ids = []
for i in range(size):
    if living_region[i]:
        ids.append(i)

In [69]:
for j in ids:
    regions = []
    position = cat_data['job_position'][j]
    salary = num_data['monthly_income'][j]
    age = num_data['age'][j]
    for i in range(size):
        f = (salary <= num_data['monthly_income'][i] + 20000 and salary >= num_data['monthly_income'][i] - 20000)
        if f and (i not in ids) and cat_data['job_position'][i] == position and age >= num_data['age'][i] - 10 and age <= num_data['age'][i] + 10:
            regions.append(cat_data['living_region'][i])
    counter = Counter(regions)
    cat_data['living_region'][j] = counter.most_common(1)[0][0]

/root/miniconda/envs/rep_py2/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [92]:
def get_binarized_data(df, categorical_columns):
    binary_df = deepcopy(df)
    for column in categorical_columns:
        binary_df = pd.concat([binary_df, pd.get_dummies(binary_df[column], prefix=column, prefix_sep=': ')], axis=1)
        del binary_df[column]
    
    return binary_df

def split_train_test(df, test_size=0.5, random_state=42, y_column_name='y'):
    X = np.asarray(df.drop(y_column_name, axis=1))
    y = np.asarray(df[y_column_name]).ravel()
    trainX, testX, trainY, testY = cross_validation.train_test_split(X, y, test_size=test_size, random_state=42)
    return trainX, testX, trainY, testY

In [83]:
bin_data = get_binarized_data(cat_data, cat_data.columns)

for column in num_data.columns:
    bin_data[column] = num_data[column]

In [93]:
trainX, testX, trainY, testY = split_train_test(df = bin_data, test_size=0.1,  y_column_name='open_account_flg')

In [107]:
for i in range(1, 11):
    reg = ExtraTreesRegressor(n_estimators=i, random_state=12, verbose=0)
    reg.fit(trainX, trainY)
    predict = reg.predict(testX)
    print roc_auc_score(testY, predict)

0.568205470185
0.600321367078
0.620963067898
0.631747897585
0.642223459821
0.652810826463
0.660167772695
0.663961276385
0.669672276713
0.672279163768


In [115]:
for i in range(50, 400, 50):
    reg = ExtraTreesRegressor(n_estimators=i, n_jobs=-1, random_state=12, verbose=0)
    reg.fit(trainX, trainY)
    predict = reg.predict(testX)
    print roc_auc_score(testY, predict)

0.701214504654
0.705820389615
0.707152754498
0.709131926797
0.709740032226
0.71004848262
0.709857664159


In [114]:
reg = ExtraTreesRegressor(n_estimators=50, n_jobs=-1, random_state=12, verbose=0)
reg.fit(trainX, trainY)
predict = reg.predict(testX)
print roc_auc_score(testY, predict)

0.701214504654


## Обработка тестовых данных

In [ ]:
test = pd.read_csv('./credit_test.csv', sep=';', parse_dates=True, dayfirst=True, encoding='cp1251', decimal = ',')
test2 = pd.read_csv('./credit_train.csv', sep=';', parse_dates=True, dayfirst=True, encoding='cp1251')

In [ ]:
cat_test = test.drop(Numerical_columns, axis = 1)
num_test = test.drop(Categorical_columns + ['client_id', 'tariff_id'], axis = 1)
num_test['tariff_id'] = test2['tariff_id']

In [ ]:
# Количество NaN в числовых признаках
print('Количество NaN:')
nan_test = num_test.isnull().sum().T
nan_test

In [ ]:
# Количество NaN в категориальных признаках
print('Количество NaN:')
nan_test2 = cat_test.isnull().sum().T
nan_test2